In [108]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.transforms as mtransforms
from pytorchtools_classifier import EarlyStopping
from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR
from matplotlib.offsetbox import AnchoredText

In [109]:
#GPU
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [171]:
class Classifier(nn.Module):

    def __init__(self,p, kernel_size):
        super(Classifier, self).__init__()
        
        self.NN = nn.Sequential(
          
            nn.Linear(48, 1000, dtype = torch.float, device = device),
            nn.BatchNorm1d(1000, dtype = torch.float),
            nn.Dropout1d(p = p),
            nn.ReLU(),
            
            nn.Linear(1000, 700, dtype = torch.float, device = device),
            nn.BatchNorm1d(700, dtype = torch.float),
            nn.Dropout1d(p = p),
            nn.ReLU(),

            nn.Linear(700, 75, dtype = torch.float, device = device),
            nn.BatchNorm1d(75, dtype = torch.float),
            nn.Dropout1d(p = p),
            nn.ReLU(),

            nn.Linear(75, 1, dtype = torch.float,  device = device),
        )
        
        #self._init_weights(self.NN)
        #nn.Sequential.apply(_init_weights)
        self.apply(self._init_weights)
    
    def _init_weights(self,module):
        
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, 0, 0.1)
            if module.bias is not None:
                module.bias.data.fill_(0.08)

    def forward(self, x):

        x = torch.flatten(x, 1)
        x = self.NN(x)

        return torch.squeeze(x, dim = 1)

In [113]:
def Reg_Loss(loss, LAMBDA):
    
    l_reg = 0
    for W in model.parameters():
        l_reg = l_reg + torch.norm(W, 1)
    
    loss_reg = loss + LAMBDA*l_reg
    
    return loss_reg

In [172]:
train, train_targets = torch.load("/home/gbortolai/Thesis/data/Jets/Classifier/dataset_train", map_location = device)
validation, validation_targets = torch.load("/home/gbortolai/Thesis/data/Jets/Classifier/dataset_validation", map_location = device)
test, test_targets = torch.load("/home/gbortolai/Thesis/data/Jets/Classifier/dataset_test", map_location = device)


train = train.to(torch.float).view(train.size(0), 1, train.size(1), train.size(2))
validation = validation.to(torch.float).view(validation.size(0), 1, validation.size(1), validation.size(2))
test = test.to(torch.float).view(test.size(0), 1, test.size(1), test.size(2))

batch_size = 100

n_batches_train = int(train.size(0)/batch_size)
train = torch.stack(torch.chunk(train, n_batches_train, dim = 0), dim = 0)
train_targets = torch.stack(torch.chunk(train_targets, n_batches_train, dim = 0), dim = 0)

In [ ]:
learing_rate = 1e-3
patience = 30
iteration = int(1e3)
LAMBDA = 1e-3
p = 0.3


model = Classifier(p, 2)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr = learing_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 30, gamma = 0.7)
early_stopping = EarlyStopping(patience = patience, verbose = True)

fig = plt.figure(figsize = (14, 7), constrained_layout=False)
gs = GridSpec(1, 2, figure = fig)

ax3 = fig.add_subplot(gs[0, 0])
ax3.set_title('Loss train average')
ax3.set_ylabel('Loss train average')
ax3.set_ybound(lower = 0, upper = None)

ax4 = fig.add_subplot(gs[0, 1])
ax4.set_title('Loss validation average')
ax4.set_ylabel('Loss validation average')
ax4.set_ybound(lower = 0, upper = None)

counter_train = 0
counter_validation = 0

train_losses = []
validation_losses = []


for ite in range(iteration):
    model.train()
    
    for batch in range(n_batches_train):
        
        train_out = model(train[batch]).to(device)
        loss_train = criterion(train_out, train_targets[batch])
        loss_train_reg = Reg_Loss(loss_train, LAMBDA)
        loss_train_reg = loss_train_reg.double()
        train_losses.append(loss_train_reg.item())
        
        optimizer.zero_grad()
        loss_train_reg.requires_grad_()
        loss_train_reg.backward(retain_graph = True)
        optimizer.step()

        
        
    ax3.scatter(ite + 1, np.average(train_losses), s = 5, color = 'b')
    train_losses = []
    
    model.eval()
    
    validation_out = model(validation).to(device)
    loss_validation = criterion(validation_out, validation_targets)
    loss_validation_reg = Reg_Loss(loss_validation, LAMBDA)
    
    ax4.scatter(ite + 1, loss_validation_reg.cpu().detach().numpy(), color = 'b', s = 5)
    
    early_stopping(loss_validation_reg, model)
    if early_stopping.early_stop:
            print("Early stopping")
            break
            
    scheduler.step()

Validation loss decreased (inf --> 27.933262).  Saving model ...
Validation loss decreased (27.933262 --> 11.353431).  Saving model ...
Validation loss decreased (11.353431 --> 6.065214).  Saving model ...
Validation loss decreased (6.065214 --> 4.349742).  Saving model ...
Validation loss decreased (4.349742 --> 3.776112).  Saving model ...
Validation loss decreased (3.776112 --> 3.196090).  Saving model ...
Validation loss decreased (3.196090 --> 3.146622).  Saving model ...
Validation loss decreased (3.146622 --> 2.904224).  Saving model ...
Validation loss decreased (2.904224 --> 2.614577).  Saving model ...
Validation loss decreased (2.614577 --> 2.510371).  Saving model ...
Validation loss decreased (2.510371 --> 2.072216).  Saving model ...
Validation loss decreased (2.072216 --> 1.923597).  Saving model ...
Validation loss decreased (1.923597 --> 1.627644).  Saving model ...
Validation loss decreased (1.627644 --> 1.412050).  Saving model ...
Validation loss decreased (1.412050

In [174]:
model.load_state_dict(torch.load("/home/gbortolai/Thesis/Checkpoint/checkpoint_classifier.pth"))

model.eval()
test_out = model(test).to(device)
loss_test = criterion(test_out, test_targets)
loss_test_reg = Reg_Loss(loss_test, LAMBDA)

for i in [1,2,3]:

    print(f"The accuracy for the class {i} is: {(torch.count_nonzero(test_out == i)/torch.count_nonzero(test_targets == i))*100}")





The accuracy for the class 1 is: 0.0
The accuracy for the class 2 is: 0.0
The accuracy for the class 3 is: 0.0


In [176]:
print(test_out)

tensor([2.1705, 2.3650, 2.3768,  ..., 2.3595, 2.3644, 1.6936],
       grad_fn=<SqueezeBackward1>)
